In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tensorflow as tf
import cv2
import seaborn as sns
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import keras
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPool2D
from keras.layers import Activation, Dropout, BatchNormalization, Flatten, Dense
from keras.optimizers import Adam
from keras.utils import to_categorical, np_utils

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder

So here below this text is where we import the files into their respective variable. We port these together to allow visualization of the data and to confirm everything has loaded in correctly.

In [ ]:
positive = [os.path.join("/kaggle/input/covid19-radiography-database/COVID-19 Radiography Database/COVID-19/",img) 
    for img in os.listdir("/kaggle/input/covid19-radiography-database/COVID-19 Radiography Database/COVID-19/")]

negative = [os.path.join("/kaggle/input/covid19-radiography-database/COVID-19 Radiography Database/NORMAL/",img) 
    for img in os.listdir("/kaggle/input/covid19-radiography-database/COVID-19 Radiography Database/NORMAL/")]

falseflag = [os.path.join("/kaggle/input/covid19-radiography-database/COVID-19 Radiography Database/Viral Pneumonia/",img) 
    for img in os.listdir("/kaggle/input/covid19-radiography-database/COVID-19 Radiography Database/Viral Pneumonia/")]

This bit of code utilized libraries, and with those libraries we 1. Selected the first photo out of the variable to know that everything has loaded in correctly. 2. Resize the photo into a 227x227 so that it can be analyzed.
Then we "plot" the photo to show that it can be plotted and show what it looks like.

In [ ]:
from keras.preprocessing.image import load_img
import matplotlib.pyplot as plt
def positiveimg(positive):
    image=load_img(positive[5],target_size=(227,227))
    plt.imshow(image)
    
positiveimg(positive)

This is the same idea except instead of doing it for the COVID-19 Positive cases we are doing it for the COVID negative cases.

In [ ]:
from keras.preprocessing.image import load_img
import matplotlib.pyplot as plt
def negativeimg(negative):
    image=load_img(negative[1],target_size=(227,227))
    plt.imshow(image)
    
negativeimg(negative)

This is going to append, or compile together all of the photos into one large image path.

In [ ]:
from keras.preprocessing.image import load_img
import matplotlib.pyplot as plt
def pneumoniaimg(falseflag):
    image=load_img(falseflag[1],target_size=(227,227))
    plt.imshow(image)
    
pneumoniaimg(falseflag)

In [ ]:
imagepath = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if (filename[-3:] == 'png'):
            imagepath.append(os.path.join(dirname, filename))

This is the code that resizes ALL of the photos, while also seperating them into their respective classes. So Pneumonia goes to Pneomonia, Normal goes to normal and COVID-19 goes to COVID-19.

In [ ]:
temp = []
temp1 = []
resize = 227

hmap = {'Viral Pneumonia': 'Pneumonia', 'NORMAL': 'Normal', 'COVID-19': 'Covid-19'}

for imagePath in tqdm(imagepath):
    label = imagePath.split(os.path.sep)[-2]
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (resize, resize))

    temp.append(image)
    temp1.append(hmap[label])

In [ ]:
le = LabelEncoder()
labels = le.fit_transform(temp1)
labels = to_categorical(labels)

Here we organize our data. So everything gets seperated into either COVID positive or negative while also seperating them into test data along with train data. We need train data, but without test data we cannot affirmatively identify the right photos.

In [ ]:
(x_train, x_test, y_train, y_test) = train_test_split(temp, labels,test_size=0.20,stratify=labels,random_state=42)

trainX = np.array(x_train)
testX = np.array(x_test)
trainY = np.array(y_train)
testY = np.array(y_test)

Here is the meat of the bone. This is our Convolution Neural Network. We are using a Sequential model, since the only thing we are doing is inputting a photo and expecting it to output back. We are going to filter it though, using ReLU activation where in the negative side its non existant, but on the positive side it does exist. And since we are using a 227x227 photo our filter will be 227x227. We are doing a 2D model so this is all that would be necessary 

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size =(3,3), activation = "relu", input_shape=(227,227,3), kernel_initializer='he_normal',))
model.add(MaxPool2D(pool_size = (2,2)))

model.add(Conv2D(filters = 64, kernel_size =(3,3), activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2)))

model.add(Conv2D(filters = 128, kernel_size =(3,3), activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2), strides = (1,1)))

model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(64, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(3, activation = "softmax"))

In [ ]:
model.compile(optimizer = keras.optimizers.Adam(), loss= keras.losses.categorical_crossentropy, metrics= ['accuracy'])

epoch= how many times we need to pass the data set throught he neural network

In [ ]:
epochs = 30
ThisModel = model.fit(trainX, trainY, epochs=epochs, batch_size=40, verbose=1)

Now we evaluate our test data, comparing our loss and accuracy

In [ ]:
modelLoss, modelAccuracy = model.evaluate(testX, testY, verbose=0)

print('Test Loss is {}'.format(modelLoss))
print('Test Accuracy is {}'.format(modelAccuracy ))

In [ ]:
class_names = ['COVID-19 Positive','COVID-19 Negative','Pneumonia']

prediction = model.predict(testX)
print(class_names[np.argmax(prediction[8])])